# Do some simple prep of compound structures from SMILES strings

In [15]:
# Make lists of the guest smiles strings and the molecule identifiers 
file = 'CB8_guest_smiles.txt'

file = open(file, 'r')
text = file.readlines()
file.close()

print("Getting list of molecule IDs...")   
MoleculeIDs = [] #SAMPL8 Molecule ID 
smiles = [] #isomeric SMILES

#Loop over lines and parse
for line in text:
    tmp = line.split(';') #Split into columns
    MoleculeID = tmp[1].strip() #mol ID
    smi = tmp[0] #smiles string
    try:
        MoleculeIDs.append(MoleculeID)
        smiles.append(smi)
    except:
        print("Error storing line: %s" % line)
print("Done!")

Getting list of molecule IDs...
Done!


## Parse SMILES, store molecules

In [16]:
from openeye.oechem import *
from openeye.oeomega import * # conformer generation
from openeye.oequacpac import * 


mols_by_ID = {}
for idx in range(len(smiles)):
    # Generate new OEMol and parse SMILES
    mol = OEMol()
    OEParseSmiles( mol, smiles[idx])
    # Set neutral pH model to pick a "reasonable" neutral protonation state per OpenEye's Quacpac
    OESetNeutralpHModel(mol)
    mols_by_ID[MoleculeIDs[idx]] = mol


## Tweak protonation states for some compounds

Some compounds seem to be coming up as neutral which likely should be protonated -- particularly, morphine (G3), pKa 8.21 (so mainly protonated at pH 7.4, https://pubchem.ncbi.nlm.nih.gov/compound/Morphine#section=Decomposition) and hydromorphone (G4), pKa 8.3 (so mainly protonated at pH 7.4, https://pubchem.ncbi.nlm.nih.gov/compound/Hydromorphone#section=Optical-Rotation). Ketamine (G5), pKa 7.5 (https://pubchem.ncbi.nlm.nih.gov/compound/Ketamine#section=Environmental-Bioconcentration) is left as is and would be a mixture of states at pH 7.4.

There's probably a more elegant way to do this (likely with a substructure search) but since G3 and G4 have only a single nitrogen I'll just loop over atoms and set the formal charge of the nitrogen to +1 and then let conformer generation take care of it

In [17]:
for mol_id in ['G4', 'G5']:
    mol = mols_by_ID[mol_id]
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum()==7:
            atom.SetFormalCharge(1)
    mols_by_ID[mol_id]=mol

# Generate conformers and assign partial charges

In [22]:


for mol_id in mols_by_ID:
    #initialize omega, this is a conformation generator
    omega = OEOmega()
    #set the maximum conformer generated to 1
    omega.SetMaxConfs(1) 
    omega.SetIncludeInput(False)
    omega.SetStrictAtomTypes(False) #Leniency in assigning atom types
    omega.SetStrictStereo(True) #Don't generate conformers if stereochemistry not provided. Setting to false would pick a random stereoisomer
    

    mol = mols_by_ID[mol_id]
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s" % (mol_id))

    # Assign charges
    OEAssignPartialCharges(mol, OECharges_AM1BCC)
        
    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()
    
    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

## A couple of molecules have unspecified stereochemistry so pick a random stereoisomer

We expect this for G5, which is a racemate. The other three -- G3, G4, and G7 -- have a nitrogen which would likely interconvert quickly in solution, but which needs stereochemistry specified for conformer generation. This may need attention from modelers, especially when there are other steroecenters.

In [23]:
# Make files for O=C1CCCCC1(NC)C2=CC=CC=C2Cl (G5) by picking a random stereoisomer, idx 4 -- this is the racemate
# Do same for G3 (idx 2), G4 (idx 4), G7 (6) -- these come up only when we set to neutral pH
# so it's likely protonation of the nitrogen?
problem_IDs = ['G3', 'G4', 'G5', 'G7']

omega = OEOmega()
#set the maximum conformer generated to 1
omega.SetMaxConfs(1) 
omega.SetIncludeInput(False)
omega.SetStrictAtomTypes(False) #Leniency in assigning atom types
omega.SetStrictStereo(False) #Don't generate conformers if stereochemistry not provided. Setting to false would pick a random stereoisomer

for mol_id in problem_IDs:
    # Generate new OEMol and parse SMILES
    mol = mols_by_ID[mol_id]
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s, %s. Mol index is %s" % (smiles[idx], MoleculeIDs[idx],idx))

    
    # Assign charges
    OEAssignPartialCharges(mol, OECharges_AM1BCC)    
    
    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(mol_id))
    OEWriteMolecule(ofile, mol)
    ofile.close()